# Simulating MSK-IMPACT 410 Gene Panel Mutational Spectra

**Goal**: simulate an MSK-IMPACT 410 gene panel mutational spectra matrix from ICGC whole genome mutational annotation file (MAF) datasets.



In [1]:
from pathlib import Path

import pandas as pd

In [2]:
dir_data = Path.cwd().parent / "data"
dir_maf_dirs = dir_data / "WGS_MAFs"

fpath_grch37 = dir_data / "hg19.fa"
fpath_msk_impact_bed = dir_data / "MSK-IMPACT410.bed"

In [11]:
dir_panel_maf_dirs = dir_data / "MSK_IMPACT_410_MAFs"
if not dir_panel_maf_dirs.exists():
        dir_panel_maf_dirs.mkdir()

In [3]:
msk_impact_df = pd.read_csv(fpath_msk_impact_bed, sep="\t")

In [4]:
msk_impact_df.head()

,Chromosome,Start_Position,End_Position,Hugo_Symbol,ID,SEQ_ASSAY_ID,Feature_Type,includeInPanel
0,9,133710831,133710915,ABL1,ABL1,MSK-IMPACT410,exon,True
1,9,133729448,133729627,ABL1,ABL1,MSK-IMPACT410,exon,True
2,9,133730185,133730486,ABL1,ABL1,MSK-IMPACT410,exon,True
3,9,133738147,133738425,ABL1,ABL1,MSK-IMPACT410,exon,True
4,9,133747513,133747603,ABL1,ABL1,MSK-IMPACT410,exon,True


In [34]:
def read_sbs_maf_file(filepath: str) -> pd.DataFrame:
    """
    Reads only single base substitutions from an MAF file generated from an
    ICGC SSM dataset.

    :param filepath: file path to the MAF file.
    :return: Pandas dataframe with only single base substitutions.
    """
    data = pd.read_csv(filepath, sep="\t")
    data = data.loc[data["mutation_type"] == "single base substitution"].reset_index(drop=True)
    return data


def is_mutation_in_gene_panel(panel_df,
                              chromosome: str,
                              position_start: int,
                              position_end: int) -> bool:
        """
        Checks if the defined mutation is measured by the gene panel.

        :param panel_df: gene panel BED file.
        :param chromosome: chromosome name of the mutation.
        :param position_start: starting position of the mutation.
        :param position_end: end position of the mutation.
        :return: True if the mutation region is measured in the gene panel.
            False otherwise.
        """
        panel_matches = panel_df.loc[((panel_df["Chromosome"] == chromosome) &
                                      (panel_df["Start_Position"] <= position_start) &
                                      (panel_df["End_Position"] >= position_end))]
        return len(panel_matches) > 0


def return_panel_rows(panel_df: pd.DataFrame,
                      wg_maf: pd.DataFrame) -> pd.DataFrame:
    """
    Takes an MAF dataframe from WGS analysis and downsamples it to only return the
    mutations covered by the gene panel (as described by its BED file).

    :param panel_df: gene panel's BED file.
    :param wg_maf: MAF from WGS analysis.
    :return:
    """
    panel_rows = wg_maf.apply(lambda row: is_mutation_in_gene_panel(panel_df,
                                                                    row["chromosome"],
                                                                    row["chromosome_start"],
                                                                    row["chromosome_end"]),
                              axis=1)
    return wg_maf.loc[panel_rows]


def convert_wg_mafs_to_panel_mafs(dir_wg_mafs: Path,
                                  panel_df: pd.DataFrame,
                                  dir_panel_mafs: Path) -> None:
        """
        Converts each MAF file, generated from WGS analysis, in a directory and
        downsamples it to only MAF files containing only the mutations covered
        by the gene panel (as described by its BED file).

        :param dir_wg_mafs: a directory containing a set of MAF files from WGS
            analysis.
        :param panel_df: BED file of the gene panel.
        :param dir_panel_mafs: output directory.
        """
        maf_filepaths = list(dir_wg_mafs.glob("*"))

        for maf_filepath in maf_filepaths:
            filepath_panel = dir_panel_mafs / f"{maf_filepath.stem}"
            data_maf = read_sbs_maf_file(maf_filepath)
            data_maf = return_panel_rows(panel_df, data_maf)
            if len(data_maf) > 0:
                data_maf.to_csv(filepath_panel)


def panel_maf_dirs_from_wgs_maf_dirs(dir_wg_maf_dirs: Path,
                                     panel_df: pd.DataFrame,
                                     dir_panel_maf_dirs: Path) -> None:
        """
        For each directory within the specified directory, this method iterates through
        all MAF files, generated from WGS analysis, and downsamples it to create MAF
        files containing mutations covered by the gene panel (as described by its BED
        file).

        :param dir_wg_maf_dirs: a directory of directories, each containing a set of MAF files
            from WGS analysis.
        :param panel_df: BED file of the gene panel.
        :param dir_panel_maf_dirs: output directory.
        """
        for dir_wg_mafs in dir_wg_maf_dirs.iterdir():
                dir_panel_mafs = dir_panel_maf_dirs / dir_wg_mafs.name
                if not dir_panel_mafs.exists():
                        dir_panel_mafs.mkdir()
                convert_wg_mafs_to_panel_mafs(dir_wg_mafs, panel_df, dir_panel_mafs)

In [ ]:
panel_maf_dirs_from_wgs_maf_dirs(dir_maf_dirs, msk_impact_df, dir_panel_maf_dirs)

/var/folders/8l/pbcwnq316rz5v5pnyv_sydmn2fsdlk/T/ipykernel_40223/3301878818.py:9: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(filepath, sep="\t")
